In [1]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import os
import dimod
from dwave.system import LeapHybridBQMSampler #LeapHybridSampler

In [2]:
# --- Generate couplings ---
def generate_J(N, seed=None):
    # Create directory if it doesn't exist
    os.makedirs('generated_data', exist_ok=True)
    
    if seed is not None:
        np.random.seed(seed)
    std = 1.0 / np.sqrt(N)
    data = [(i, j, np.random.normal(0, std)) for i in range(N) for j in range(i+1, N)]
    df = pd.DataFrame(data, columns=['i','j','Jij'])
    
    if seed is not None:
        filepath = f'generated_data/Jij_N{N}_seed{seed}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")
    else:
        filepath = f'generated_data/Jij_N{N}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")

# --- Generate perturbed couplings (Goal 3) ---
def generate_J_perturbed(N, seed_orig, seed_pert, epsilon=0.01):
    """
    Generate perturbed couplings: J_ij^pert = J_ij^orig + epsilon * delta_J_ij
    where delta_J_ij is independent noise with same distribution as J_ij.
    """
    os.makedirs('generated_data', exist_ok=True)
    
    # Load original couplings
    df_orig = pd.read_csv(f'generated_data/Jij_N{N}_seed{seed_orig}.csv')
    
    # Generate independent perturbation noise
    np.random.seed(seed_pert)
    std = 1.0 / np.sqrt(N)
    delta_J = [np.random.normal(0, std) for _ in range(len(df_orig))]
    
    # Create perturbed couplings
    df_pert = df_orig.copy()
    df_pert['Jij'] = df_orig['Jij'] + epsilon * np.array(delta_J)
    
    filepath = f'generated_data/Jij_N{N}_seed{seed_orig}_pert{seed_pert}_eps{epsilon}.csv'
    df_pert.to_csv(filepath, index=False)
    print(f"Saved perturbed couplings to {filepath}")
    return df_pert

# --- Load couplings ---
def load_J(N, seed=None, seed_pert=None, epsilon=0.01):
    if seed_pert is not None:
        # Load perturbed couplings
        filename = f'generated_data/Jij_N{N}_seed{seed}_pert{seed_pert}_eps{epsilon}.csv'
    elif seed is not None:
        # Load original couplings
        filename = f'generated_data/Jij_N{N}_seed{seed}.csv'
    else:
        filename = f'generated_data/Jij_N{N}.csv'
    
    df = pd.read_csv(filename)
    N = max(df['i'].max(), df['j'].max()) + 1
    return [(int(i), int(j), float(Jij)) for i,j,Jij in df.values], N

In [4]:
# --- Build and solve D-Wave BQM model ---
def solve_SK_dwave(rows, N, seed=None, time_limit=30, perturbed=False, seed_pert=None, epsilon=0.01):
    # Create directory if it doesn't exist
    os.makedirs('dwave_solutions', exist_ok=True)
    # Build Binary Quadratic Model (BQM) in SPIN variables
    h = {i: 0.0 for i in range(N)} # no onsite field
    J = {(i, j): -Jij for i, j, Jij in rows} # match SK Hamiltonian: H = -sum Jij S_i S_j

    bqm = dimod.BinaryQuadraticModel(h, J, dimod.SPIN)

    sampler = LeapHybridBQMSampler() #LeapHybridSampler()
    sampleset = sampler.sample(bqm, time_limit=time_limit)

    result = sampleset.first.sample
    S = [result[i] for i in range(N)]
    energy = min(sampleset.record.energy)

    print(f"N = {N}\nEnergy = {energy}") 
    print(f"First spins: {S[:10]}")  # Show first 10 spins only

    # Save with appropriate filename
    if perturbed and seed_pert is not None:
        filepath = f'dwave_solutions/solution_N{N}_seed{seed}_pert{seed_pert}_eps{epsilon}.csv'
    else:
        filepath = f'dwave_solutions/solution_N{N}_seed{seed}.csv'

    pd.DataFrame({'i': range(N), 'Energy_dwave': energy, 'S': S}).to_csv(filepath, index=False)
    print(f"Saved solution to {filepath}")
    
    return energy, S

## Goal 3: Chaos Exponent - Load Original Ground States

Load existing ground state energies from Phase 1 (no need to recompute)

In [5]:
# Goal 3: Load original ground state energies (already computed in Phase 1)
N = 500
M = 20  # Use first 20 seeds from Phase 1

# Phase 1 used seeds: i*5 for i in range(1, 41) → [5, 10, 15, ..., 200]
# We use first 20: i*5 for i in range(1, 21) → [5, 10, 15, ..., 100]

energy_orig_list = []
original_seeds = []

for i in range(1, M+1):
    seed = i * 5
    original_seeds.append(seed)
    
    filepath = f'dwave_solutions/solution_N{N}_seed{seed}.csv'
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        E_orig = df['Energy_dwave'][0]
        print(f"Loaded existing solution for seed {seed}: E = {E_orig:.4f}")
        energy_orig_list.append(E_orig)
    else:
        print(f"ERROR: Missing solution file for seed {seed}")

print(f"\nLoaded {len(energy_orig_list)} original energies.")
print(f"Seeds used: {original_seeds}")
print(f"Mean E/N = {np.mean(energy_orig_list)/N:.4f}")

Loaded existing solution for seed 5: E = -377.4225
Loaded existing solution for seed 10: E = -375.3102
Loaded existing solution for seed 15: E = -375.8972
Loaded existing solution for seed 20: E = -376.8542
Loaded existing solution for seed 25: E = -372.0149
Loaded existing solution for seed 30: E = -384.0999
Loaded existing solution for seed 35: E = -376.6050
Loaded existing solution for seed 40: E = -374.3880
Loaded existing solution for seed 45: E = -371.7367
Loaded existing solution for seed 50: E = -371.7920
Loaded existing solution for seed 55: E = -375.6804
Loaded existing solution for seed 60: E = -372.3152
Loaded existing solution for seed 65: E = -375.6448
Loaded existing solution for seed 70: E = -372.5616
Loaded existing solution for seed 75: E = -375.2747
Loaded existing solution for seed 80: E = -379.3933
Loaded existing solution for seed 85: E = -374.1100
Loaded existing solution for seed 90: E = -373.1405
Loaded existing solution for seed 95: E = -373.9728
Loaded existi

## Goal 3: Perturbed Ground States

Generate perturbed couplings and compute ground states with D-Wave

In [7]:
# original_seeds

In [8]:
# Goal 3: Compute perturbed ground states
# Option A: M=20 disorder realizations, each with independent perturbation
epsilon = 0.01

energy_pert_list = []
delta_E_list = []

for idx, seed_orig in enumerate(original_seeds):
    # Use different perturbation seed for each disorder (seed_pert = 10000 + seed_orig)
    seed_pert = 10000 + seed_orig
    
    print(f"\n--- Processing seed {seed_orig} (sample {idx+1}/{M}) ---")
    
    # Generate perturbed couplings
    generate_J_perturbed(N, seed_orig=seed_orig, seed_pert=seed_pert, epsilon=epsilon)
    
    # Load and solve perturbed instance
    rows_pert, N = load_J(N, seed=seed_orig, seed_pert=seed_pert, epsilon=epsilon)
    E_pert, S_pert = solve_SK_dwave(rows_pert, N, seed=seed_orig, time_limit=45, 
                                     perturbed=True, seed_pert=seed_pert, epsilon=epsilon)
    
    energy_pert_list.append(E_pert)
    
    # Compute energy change
    E_orig = energy_orig_list[idx]
    delta_E = abs(E_pert - E_orig)
    delta_E_list.append(delta_E)
    
    print(f"Seed {seed_orig}, Pert {seed_pert}: E_orig = {E_orig:.4f}, E_pert = {E_pert:.4f}, |ΔE| = {delta_E:.4f}")

# Compute RMS energy change (disorder-averaged)
delta_E_rms = np.sqrt(np.mean(np.array(delta_E_list)**2))
print(f"\n{'='*60}")
print(f"--- Goal 3 Results (N={N}, ε={epsilon}) ---")
print(f"RMS energy change: ⟨(ΔE)²⟩^(1/2) = {delta_E_rms:.4f}")
print(f"Mean |ΔE| = {np.mean(delta_E_list):.4f} ± {np.std(delta_E_list):.4f}")
print(f"Standard protocol: {M} disorder realizations, each with independent perturbation")
print(f"{'='*60}")


--- Processing seed 5 (sample 1/20) ---
Saved perturbed couplings to generated_data/Jij_N500_seed5_pert10005_eps0.01.csv
N = 500
Energy = -377.3012598105194
First spins: [1, -1, 1, -1, -1, -1, 1, 1, 1, 1]
Saved solution to dwave_solutions/solution_N500_seed5_pert10005_eps0.01.csv
Seed 5, Pert 10005: E_orig = -377.4225, E_pert = -377.3013, |ΔE| = 0.1212

--- Processing seed 10 (sample 2/20) ---
Saved perturbed couplings to generated_data/Jij_N500_seed10_pert10010_eps0.01.csv
N = 500
Energy = -375.2462400764836
First spins: [-1, 1, 1, 1, -1, 1, 1, 1, 1, 1]
Saved solution to dwave_solutions/solution_N500_seed10_pert10010_eps0.01.csv
Seed 10, Pert 10010: E_orig = -375.3102, E_pert = -375.2462, |ΔE| = 0.0639

--- Processing seed 15 (sample 3/20) ---
Saved perturbed couplings to generated_data/Jij_N500_seed15_pert10015_eps0.01.csv
N = 500
Energy = -375.9275900802428
First spins: [-1, 1, -1, 1, -1, -1, 1, 1, -1, -1]
Saved solution to dwave_solutions/solution_N500_seed15_pert10015_eps0.01.csv

In [10]:
# Save Goal 3 results for chaos exponent analysis
results_df = pd.DataFrame({
    'seed_orig': original_seeds,
    'seed_pert': [10000 + s for s in original_seeds],
    'E_orig': energy_orig_list,
    'E_pert': energy_pert_list,
    'delta_E': delta_E_list
})

os.makedirs('chaos_results', exist_ok=True)
results_df.to_csv(f'chaos_results/chaos_N{N}_eps{epsilon}.csv', index=False)
print(f"\nSaved chaos results to chaos_results/chaos_N{N}_eps{epsilon}.csv")
print(f"Protocol: {M} disorder realizations × 1 perturbation each = {M} samples")
print(f"Original seeds from Phase 1: {original_seeds}")


Saved chaos results to chaos_results/chaos_N500_eps0.01.csv
Protocol: 20 disorder realizations × 1 perturbation each = 20 samples
Original seeds from Phase 1: [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
